In [3]:
%load_ext autoreload

In [4]:
import pandas as pd
import numpy as np
import os
import feather
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error 
import random
import csv

In [5]:
import sys
if "/proj/yunligrp/users/minzhi/utils/pylib" not in sys.path:
    sys.path.insert(0, "/proj/yunligrp/users/minzhi/utils/pylib")

In [6]:
from function_process_data_eqtl import *
from function_asso import *
from function_mesa_cca import *
%autoreload 2

## Blood Cell Traits

#### ARIC

In [67]:
aric_bct_dir = "/proj/yunligrp/dbGAP_topmed/dbGaP-12526/57456/topmed-dcc/exchange/phs001211_TOPMed_WGS_ARIC/Phenotype/DCC/official/topmed_dcc_blood_cell_count_20170414_phs001211/"
aric_bct_filename = "topmed_dcc_blood_cell_count_20170414.txt"
aric_bct_dir_filename = os.path.join(aric_bct_dir, aric_bct_filename)
aric_bct = pd.read_csv(aric_bct_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [68]:
aric_bct_prepared = aric_bct.copy()
aric_bct_prepared["cohort"] = "ARIC"
aric_bct_prepared["unique_subject_key"] = aric_bct_prepared["cohort"] + "_" + aric_bct_prepared["SUBJECT_ID"]
aric_bct_prepared_dir = os.path.join("..", "raw_data", "phenotype")
aric_bct_prepared_filename = "ARIC_blood_cell_traits.tsv"
aric_bct_prepared_dir_filename = os.path.join(aric_bct_prepared_dir, aric_bct_prepared_filename)
aric_bct_prepared.to_csv(aric_bct_prepared_dir_filename, sep = "\t", header = True, index = False)

In [195]:
dg_dir = "/proj/yunligrp/users/raffield/TOPMED/Hematology_variants/ABOvariants/freeze5/topmed_dcc_demographic_v2_phs001211_aric/"
dg_filename = "topmed_dcc_demographic_v2.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [26]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
1,Black,3896
0,White,11044


,ethnicity_1,sample_size
0,Missing Values,14940


In [196]:
dg_useful = dg[["unique_subject_key", "race_1"]]
dg_useful.rename(columns = {"race_1":"race"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1}, inplace = True)
dg_useful["ethnicity"] = 0
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "ARIC_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,unique_subject_key,race,ethnicity
0,ARIC_131055557,2,0
1,ARIC_131159762,2,0
2,ARIC_131158564,2,0
3,ARIC_131109096,1,0
4,ARIC_131038426,1,0
...,...,...,...
14935,ARIC_131146794,1,0
14936,ARIC_131144932,1,0
14937,ARIC_131177858,2,0
14938,ARIC_131097159,2,0


In [12]:
display(aric_dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1,geographic_site_1
0,131055557,ARIC_131055557,ARIC,Black,NaN,NaN,male,ARIC_C
1,131159762,ARIC_131159762,ARIC,Black,NaN,NaN,female,ARIC_C
2,131158564,ARIC_131158564,ARIC,Black,NaN,NaN,female,ARIC_C
3,131109096,ARIC_131109096,ARIC,White,NaN,NaN,male,ARIC_B
4,131038426,ARIC_131038426,ARIC,White,NaN,NaN,female,ARIC_B
...,...,...,...,...,...,...,...,...
14935,131146794,ARIC_131146794,ARIC,White,NaN,NaN,female,ARIC_B
14936,131144932,ARIC_131144932,ARIC,White,NaN,NaN,male,ARIC_B
14937,131177858,ARIC_131177858,ARIC,Black,NaN,NaN,male,ARIC_C
14938,131097159,ARIC_131097159,ARIC,Black,NaN,NaN,male,ARIC_C


#### SAFS

In [48]:
safs_bct_dir = "/proj/yunligrp/dbGAP_topmed/longleaf/dbGaP-12526/74709/topmed-dcc/exchange/phs001215_TOPMed_WGS_SAFHS_CVD/Phenotype/DCC/other/SAFS"
safs_bct_filename = "unofficial_topmed_dcc_blood_cell_count_2018_01_19.txt"
safs_bct_dir_filename = os.path.join(safs_bct_dir, safs_bct_filename)
safs_bct = pd.read_csv(safs_bct_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [49]:
safs_bct.rename(columns = {"topmed_abbreviation":"cohort"}, inplace = True)
safs_bct.drop(columns = ["granulocyte_ncnc_bld_1", "age_at_granulocyte_ncnc_bld_1"], inplace = True)
safs_bct_dir = os.path.join("..", "raw_data", "phenotype")
safs_bct_filename = "SAFS_blood_cell_traits.tsv"
safs_bct_dir_filename = os.path.join(safs_bct_dir, safs_bct_filename)
safs_bct.to_csv(safs_bct_dir_filename, sep = "\t", header = True, index = False)

In [46]:
display(safs_bct)

,SUBJECT_ID,unique_subject_key,age_at_pmv_entvol_bld_1,pmv_entvol_bld_1,age_at_hematocrit_vfr_bld_1,hematocrit_vfr_bld_1,age_at_hemoglobin_mcnc_bld_1,hemoglobin_mcnc_bld_1,age_at_platelet_ncnc_bld_1,platelet_ncnc_bld_1,...,mch_entmass_rbc_1,age_at_mchc_mcnc_rbc_1,mchc_mcnc_rbc_1,age_at_mcv_entvol_rbc_1,mcv_entvol_rbc_1,age_at_rdw_ratio_rbc_1,rdw_ratio_rbc_1,age_at_rbc_ncnc_bld_1,rbc_ncnc_bld_1,cohort
0,DI00001,SAFS_DI00001,41.98,9.133333,41.98,39.133333,41.98,12.833333,41.98,275.000000,...,26.666667,41.98,32.900000,41.98,81.100000,NaN,NaN,41.98,4.820000,SAFS
1,DI00002,SAFS_DI00002,50.02,7.675000,50.02,35.700000,50.02,11.525000,50.02,247.250000,...,26.900000,50.02,32.250000,50.02,83.475000,NaN,NaN,50.02,4.280000,SAFS
2,DI00003,SAFS_DI00003,53.73,8.200000,53.73,33.666667,53.73,10.866667,53.73,411.666667,...,25.566667,53.73,32.233333,53.73,79.300000,NaN,NaN,53.73,4.243333,SAFS
3,DI00004,SAFS_DI00004,56.48,10.466667,56.48,33.133333,56.48,10.933333,56.48,190.666667,...,26.333333,56.48,33.066667,56.48,79.633333,NaN,NaN,56.48,4.163333,SAFS
4,DI00005,SAFS_DI00005,46.49,8.333333,46.49,37.133333,46.49,12.766667,46.49,174.666667,...,32.600000,46.49,34.366667,46.49,94.933333,NaN,NaN,46.49,3.916667,SAFS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,DI01626,SAFS_DI01626,59.20,9.600000,59.20,40.133333,59.20,13.666667,59.20,168.333333,...,32.000000,59.20,34.100000,59.20,93.800000,NaN,NaN,59.20,4.276667,SAFS
1490,DI01856,SAFS_DI01856,41.80,9.100000,41.80,40.900000,41.80,13.633333,41.80,249.333333,...,30.600000,41.80,33.300000,41.80,91.966667,NaN,NaN,41.80,4.446667,SAFS
1491,DI01866,SAFS_DI01866,40.38,8.033333,40.38,43.133333,40.38,14.666667,40.38,230.333333,...,30.266667,40.38,34.033333,40.38,88.833333,NaN,NaN,40.38,4.856667,SAFS
1492,DI01911,SAFS_DI01911,22.24,8.000000,22.24,39.100000,22.24,13.300000,NaN,NaN,...,29.533333,22.24,34.000000,22.24,86.800000,NaN,NaN,22.24,4.500000,SAFS


In [5]:
safs_dg_dir = "/proj/yunligrp/dbGAP_topmed/longleaf/dbGaP-12526/74709/topmed-dcc/exchange/phs001215_TOPMed_WGS_SAFHS_CVD/Phenotype/DCC/other/unofficial_topmed_dcc_demographic_2018-11-29_phs001215"
safs_dg_filename = "unofficial_topmed_dcc_demographic_2018-11-29.txt"
safs_dg_dir_filename = os.path.join(safs_dg_dir, safs_dg_filename)
safs_dg = pd.read_csv(safs_dg_dir_filename, sep = "\t", header = 0, index_col = None)

In [10]:
safs_dg.dropna(subset = ["race_1"], how = "any", inplace = True)
display(safs_dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1


In [23]:
display(safs_bct)

,SUBJECT_ID,unique_subject_key,age_at_pmv_entvol_bld_1,pmv_entvol_bld_1,age_at_hematocrit_vfr_bld_1,hematocrit_vfr_bld_1,age_at_hemoglobin_mcnc_bld_1,hemoglobin_mcnc_bld_1,age_at_platelet_ncnc_bld_1,platelet_ncnc_bld_1,...,mchc_mcnc_rbc_1,age_at_mcv_entvol_rbc_1,mcv_entvol_rbc_1,age_at_rdw_ratio_rbc_1,rdw_ratio_rbc_1,age_at_rbc_ncnc_bld_1,rbc_ncnc_bld_1,topmed_abbreviation,granulocyte_ncnc_bld_1,age_at_granulocyte_ncnc_bld_1
0,DI00001,SAFS_DI00001,41.98,9.133333,41.98,39.133333,41.98,12.833333,41.98,275.000000,...,32.900000,41.98,81.100000,NaN,NaN,41.98,4.820000,SAFS,3.500489,41.98
1,DI00002,SAFS_DI00002,50.02,7.675000,50.02,35.700000,50.02,11.525000,50.02,247.250000,...,32.250000,50.02,83.475000,NaN,NaN,50.02,4.280000,SAFS,3.312344,50.02
2,DI00003,SAFS_DI00003,53.73,8.200000,53.73,33.666667,53.73,10.866667,53.73,411.666667,...,32.233333,53.73,79.300000,NaN,NaN,53.73,4.243333,SAFS,4.605867,53.73
3,DI00004,SAFS_DI00004,56.48,10.466667,56.48,33.133333,56.48,10.933333,56.48,190.666667,...,33.066667,56.48,79.633333,NaN,NaN,56.48,4.163333,SAFS,4.668167,56.48
4,DI00005,SAFS_DI00005,46.49,8.333333,46.49,37.133333,46.49,12.766667,46.49,174.666667,...,34.366667,46.49,94.933333,NaN,NaN,46.49,3.916667,SAFS,4.373111,46.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,DI01626,SAFS_DI01626,59.20,9.600000,59.20,40.133333,59.20,13.666667,59.20,168.333333,...,34.100000,59.20,93.800000,NaN,NaN,59.20,4.276667,SAFS,NaN,NaN
1490,DI01856,SAFS_DI01856,41.80,9.100000,41.80,40.900000,41.80,13.633333,41.80,249.333333,...,33.300000,41.80,91.966667,NaN,NaN,41.80,4.446667,SAFS,NaN,NaN
1491,DI01866,SAFS_DI01866,40.38,8.033333,40.38,43.133333,40.38,14.666667,40.38,230.333333,...,34.033333,40.38,88.833333,NaN,NaN,40.38,4.856667,SAFS,NaN,NaN
1492,DI01911,SAFS_DI01911,22.24,8.000000,22.24,39.100000,22.24,13.300000,NaN,NaN,...,34.000000,22.24,86.800000,NaN,NaN,22.24,4.500000,SAFS,NaN,NaN


#### COPDGene

In [186]:
dg_dir = "/proj/yunligrp/users/raffield/TOPMED/Hematology_variants/ABOvariants/phen_freeze6/COPDGene/topmed_dcc_demographic_v3_phs000951"
dg_filename = "topmed_dcc_demographic_v3.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [29]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
0,Black,3438
1,White,6933


,ethnicity_1,sample_size
0,notHL,10371


In [187]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1}, inplace = True)
dg_useful["ethnicity"] = 0
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "COPDGene_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,unique_subject_key,race,ethnicity
0,COPDGene_COPDGene_U03468,1,0
1,COPDGene_COPDGene_V00352,1,0
2,COPDGene_COPDGene_D50111,1,0
3,COPDGene_COPDGene_X09897,1,0
4,COPDGene_COPDGene_Q16188,2,0
...,...,...,...
10366,COPDGene_COPDGene_N88440,2,0
10367,COPDGene_COPDGene_M74765,1,0
10368,COPDGene_COPDGene_T72874,2,0
10369,COPDGene_COPDGene_V16400,2,0


In [6]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1,geographic_site_1,subcohort_1
0,COPDGene_U03468,COPDGene_COPDGene_U03468,COPDGene,White,notHL,NaN,male,COPDGene_C03,COPDGene_NoSubcohortStructure
1,COPDGene_V00352,COPDGene_COPDGene_V00352,COPDGene,White,notHL,NaN,male,COPDGene_C16,COPDGene_NoSubcohortStructure
2,COPDGene_D50111,COPDGene_COPDGene_D50111,COPDGene,White,notHL,NaN,female,COPDGene_C02,COPDGene_NoSubcohortStructure
3,COPDGene_X09897,COPDGene_COPDGene_X09897,COPDGene,White,notHL,NaN,male,COPDGene_C11,COPDGene_NoSubcohortStructure
4,COPDGene_Q16188,COPDGene_COPDGene_Q16188,COPDGene,Black,notHL,NaN,female,COPDGene_C03,COPDGene_NoSubcohortStructure
...,...,...,...,...,...,...,...,...,...
10366,COPDGene_N88440,COPDGene_COPDGene_N88440,COPDGene,Black,notHL,NaN,male,COPDGene_C03,COPDGene_NoSubcohortStructure
10367,COPDGene_M74765,COPDGene_COPDGene_M74765,COPDGene,White,notHL,NaN,male,COPDGene_C08,COPDGene_NoSubcohortStructure
10368,COPDGene_T72874,COPDGene_COPDGene_T72874,COPDGene,Black,notHL,NaN,male,COPDGene_C06,COPDGene_NoSubcohortStructure
10369,COPDGene_V16400,COPDGene_COPDGene_V16400,COPDGene,Black,notHL,NaN,male,COPDGene_C13,COPDGene_NoSubcohortStructure


#### BioMe

In [184]:
dg_dir = "/proj/yunligrp/dbGAP_topmed/longleaf/dbGaP-12526/68503/topmed-dcc/exchange/phs001644_TOPMed_WGS_BioME/Phenotype/DCC/other/unofficial_topmed_dcc_demographic_2018-11-29_phs001644"
dg_filename = "unofficial_topmed_dcc_demographic_2018-11-29.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [32]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
2,AI_AN,28
1,Asian,337
3,Black,3917
0,Missing Values,5840
4,Other,1066
5,White,4402


,ethnicity_1,sample_size
1,HL,5840
0,Missing Values,9750


In [185]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1, "Asian":3, "AI_AN":4, "Other":6, "HL":1}, inplace = True)
sample_num = dg_useful.shape[0]
for sample_i in range(sample_num):
    if pd.isnull(dg_useful.loc[sample_i, "race"]) == True and dg_useful.loc[sample_i, "ethnicity"] == 1:
#         display(dg_useful.iloc[sample_i, :])
        dg_useful.loc[sample_i, "race"] = 0
    elif pd.isnull(dg_useful.loc[sample_i, "race"]) == False and pd.isnull(dg_useful.loc[sample_i, "ethnicity"]) == True:
        dg_useful.loc[sample_i, "ethnicity"] = 0
#     if sample_i % 500 == 0:
#         print(f"{sample_i + 1}/{sample_num} finished.")

# print(dg_useful.shape)
dg_useful.dropna(axis = 0, how = "any", inplace = True)
# display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "BioMe_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [8]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1
0,TPMCCDG18269,BioMe_TPMCCDG18269,BioMe,NaN,HL,NaN,female
1,TPMCCDG6176,BioMe_TPMCCDG6176,BioMe,White,NaN,NaN,female
2,TPMCCDG13911,BioMe_TPMCCDG13911,BioMe,White,NaN,NaN,male
3,TPMCCDG18938,BioMe_TPMCCDG18938,BioMe,NaN,HL,NaN,male
4,TPMCCDG19581,BioMe_TPMCCDG19581,BioMe,Other,NaN,NaN,male
...,...,...,...,...,...,...,...
15585,TPMCCDG6083,BioMe_TPMCCDG6083,BioMe,Black,NaN,NaN,female
15586,TPMCCDG9785,BioMe_TPMCCDG9785,BioMe,White,NaN,NaN,male
15587,TPMCCDG14485,BioMe_TPMCCDG14485,BioMe,Black,NaN,NaN,male
15588,TPMCCDG8964,BioMe_TPMCCDG8964,BioMe,White,NaN,NaN,female


#### GeneSTAR

In [182]:
dg_dir = "/proj/yunligrp/dbGAP_topmed/longleaf/dbGaP-12526/76122/topmed-dcc/exchange/phs001218_TOPMed_WGS_GeneSTAR/Phenotype/DCC/official/topmed_dcc_harmonized_demographic_v4_phs001218"
dg_filename = "topmed_dcc_harmonized_demographic_v4.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [34]:
race_summary_df, _ = categorize_df(dg, "race_us_1")
display(race_summary_df)
# ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
# display(ethnicity_summary_df)

,race_us_1,sample_size
0,Black,786
1,White,1001


In [183]:
dg_useful = dg[["unique_subject_key", "race_us_1"]]
dg_useful.rename(columns = {"race_us_1":"race"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1}, inplace = True)
dg_useful["ethnicity"] = 0
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "GeneSTAR_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,unique_subject_key,race,ethnicity
0,GeneSTAR_GS75606558,1,0
1,GeneSTAR_GS86065180,2,0
2,GeneSTAR_GS68879129,1,0
3,GeneSTAR_GS00464430,2,0
4,GeneSTAR_GS76699707,1,0
...,...,...,...
1782,GeneSTAR_GS78334807,2,0
1783,GeneSTAR_GS38267623,2,0
1784,GeneSTAR_GS11508024,2,0
1785,GeneSTAR_GS78837526,2,0


In [10]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_study,hispanic_subgroup_1,unit_hispanic_subgroup_1,annotated_sex_1,unit_annotated_sex_1,geographic_site_1,unit_geographic_site_1,subcohort_1,unit_subcohort_1,hispanic_or_latino_1,unit_hispanic_or_latino_1,race_us_1,unit_race_us_1
0,GS75606558,GeneSTAR_GS75606558,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,White,GeneSTAR
1,GS86065180,GeneSTAR_GS86065180,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,Black,GeneSTAR
2,GS68879129,GeneSTAR_GS68879129,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,White,GeneSTAR
3,GS00464430,GeneSTAR_GS00464430,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,Black,GeneSTAR
4,GS76699707,GeneSTAR_GS76699707,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_CoparentOffspring,GeneSTAR,NaN,NaN,White,GeneSTAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,GS78334807,GeneSTAR_GS78334807,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,Black,GeneSTAR
1783,GS38267623,GeneSTAR_GS38267623,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_CoparentOffspring,GeneSTAR,NaN,NaN,Black,GeneSTAR
1784,GS11508024,GeneSTAR_GS11508024,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,Black,GeneSTAR
1785,GS78837526,GeneSTAR_GS78837526,GeneSTAR,NaN,NaN,female,GeneSTAR,NaN,NaN,GeneSTAR_Sibling,GeneSTAR,NaN,NaN,Black,GeneSTAR


#### CARDIA

In [180]:
dg_dir = "/proj/yunligrp/dbGAP_topmed/dbGaP-12526/64890/topmed-dcc/exchange/phs001612_TOPMed_WGS_CARDIA/Phenotype/DCC/official/topmed_dcc_demographic_v3_phs001612"
dg_filename = "topmed_dcc_demographic_v3.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [36]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
0,Black,1680
1,White,1942


,ethnicity_1,sample_size
0,Missing Values,3622


In [181]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1}, inplace = True)
dg_useful["ethnicity"] = 0
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "CARDIA_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,unique_subject_key,race,ethnicity
0,CARDIA_221192454,1,0
1,CARDIA_221109394,2,0
2,CARDIA_221158338,1,0
3,CARDIA_221005619,1,0
4,CARDIA_221021832,2,0
...,...,...,...
3617,CARDIA_221007232,1,0
3618,CARDIA_221165140,1,0
3619,CARDIA_221020310,2,0
3620,CARDIA_221047642,1,0


In [12]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1,geographic_site_1,subcohort_1
0,221192454,CARDIA_221192454,CARDIA,White,NaN,NaN,male,CARDIA_4,CARDIA_NoSubcohortStructure
1,221109394,CARDIA_221109394,CARDIA,Black,NaN,NaN,male,CARDIA_4,CARDIA_NoSubcohortStructure
2,221158338,CARDIA_221158338,CARDIA,White,NaN,NaN,female,CARDIA_4,CARDIA_NoSubcohortStructure
3,221005619,CARDIA_221005619,CARDIA,White,NaN,NaN,male,CARDIA_3,CARDIA_NoSubcohortStructure
4,221021832,CARDIA_221021832,CARDIA,Black,NaN,NaN,female,CARDIA_4,CARDIA_NoSubcohortStructure
...,...,...,...,...,...,...,...,...,...
3617,221007232,CARDIA_221007232,CARDIA,White,NaN,NaN,male,CARDIA_4,CARDIA_NoSubcohortStructure
3618,221165140,CARDIA_221165140,CARDIA,White,NaN,NaN,male,CARDIA_4,CARDIA_NoSubcohortStructure
3619,221020310,CARDIA_221020310,CARDIA,Black,NaN,NaN,male,CARDIA_1,CARDIA_NoSubcohortStructure
3620,221047642,CARDIA_221047642,CARDIA,White,NaN,NaN,female,CARDIA_1,CARDIA_NoSubcohortStructure


#### HCHS_SOL

In [178]:
dg_dir = "/proj/yunligrp/users/raffield/TOPMED/Hematology_variants/ABOvariants/phen_freeze6/HCHS_SOL/topmed_dcc_demographic_v3_phs001395"
dg_filename = "topmed_dcc_demographic_v3.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [38]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
0,Other,12895


,ethnicity_1,sample_size
0,HL,12895


In [179]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Other":0, "HL":1}, inplace = True)
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "HCHS_SOL_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,unique_subject_key,race,ethnicity
0,HCHS_SOL_SoL557266,0,1
1,HCHS_SOL_SoL305090,0,1
2,HCHS_SOL_SoL242966,0,1
3,HCHS_SOL_SoL114516,0,1
4,HCHS_SOL_SoL999927,0,1
...,...,...,...
12890,HCHS_SOL_SoL562694,0,1
12891,HCHS_SOL_SoL419929,0,1
12892,HCHS_SOL_SoL477268,0,1
12893,HCHS_SOL_SoL853833,0,1


In [14]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1,geographic_site_1,subcohort_1
0,SoL557266,HCHS_SOL_SoL557266,HCHS_SOL,Other,HL,Mexican,female,HCHS_SOL_SanDiego,NaN
1,SoL305090,HCHS_SOL_SoL305090,HCHS_SOL,Other,HL,Cuban,male,HCHS_SOL_Miami,NaN
2,SoL242966,HCHS_SOL_SoL242966,HCHS_SOL,Other,HL,NaN,female,HCHS_SOL_Chicago,NaN
3,SoL114516,HCHS_SOL_SoL114516,HCHS_SOL,Other,HL,NaN,male,HCHS_SOL_Chicago,NaN
4,SoL999927,HCHS_SOL_SoL999927,HCHS_SOL,Other,HL,Mexican,male,HCHS_SOL_SanDiego,NaN
...,...,...,...,...,...,...,...,...,...
12890,SoL562694,HCHS_SOL_SoL562694,HCHS_SOL,Other,HL,PuertoRican,female,HCHS_SOL_Chicago,NaN
12891,SoL419929,HCHS_SOL_SoL419929,HCHS_SOL,Other,HL,PuertoRican,male,HCHS_SOL_Bronx,NaN
12892,SoL477268,HCHS_SOL_SoL477268,HCHS_SOL,Other,HL,NaN,male,HCHS_SOL_SanDiego,NaN
12893,SoL853833,HCHS_SOL_SoL853833,HCHS_SOL,Other,HL,Mexican,male,HCHS_SOL_SanDiego,NaN


#### CHS

In [14]:
dg_dir = "/proj/yunligrp/users/raffield/TOPMED/Hematology_variants/ABOvariants/phen_freeze6/CHS/topmed_dcc_demographic_v3_phs001368"
dg_filename = "topmed_dcc_demographic_v3.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [15]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
1,AI_AN,14
0,Asian,4
4,Black,842
2,Other,18
3,White,4653


,ethnicity_1,sample_size
2,HL,56
0,Missing Values,20
1,notHL,5455


In [16]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1, "Asian":3, "AI_AN":4, "Other":6, "HL":1, "notHL":0}, inplace = True)

sample_num = dg_useful.shape[0]
for sample_i in range(sample_num):
    if pd.isnull(dg_useful.loc[sample_i, "race"]) == False and pd.isnull(dg_useful.loc[sample_i, "ethnicity"]) == True:
        dg_useful.loc[sample_i, "ethnicity"] = 0
        
print(dg_useful.shape)
dg_useful.dropna(axis = 0, how = "any", inplace = True)
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "CHS_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

(5531, 3)


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,unique_subject_key,race,ethnicity
0,CHS_956534,1,0.0
1,CHS_329089,1,0.0
2,CHS_821686,1,0.0
3,CHS_594685,1,0.0
4,CHS_594737,1,0.0
...,...,...,...
5526,CHS_524223,1,0.0
5527,CHS_823280,1,0.0
5528,CHS_203847,1,0.0
5529,CHS_405396,1,0.0


In [16]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_abbreviation,race_1,ethnicity_1,hispanic_subgroup_1,annotated_sex_1,geographic_site_1,subcohort_1
0,956534,CHS_956534,CHS,White,notHL,NaN,female,CHS_PITT,CHS_Original
1,329089,CHS_329089,CHS,White,notHL,NaN,female,CHS_DAVIS,CHS_Original
2,821686,CHS_821686,CHS,White,notHL,NaN,female,CHS_PITT,CHS_Original
3,594685,CHS_594685,CHS,White,notHL,NaN,male,CHS_BOWMAN.GRAY,CHS_Original
4,594737,CHS_594737,CHS,White,notHL,NaN,male,CHS_PITT,CHS_Original
...,...,...,...,...,...,...,...,...,...
5526,524223,CHS_524223,CHS,White,notHL,NaN,male,CHS_DAVIS,CHS_Original
5527,823280,CHS_823280,CHS,White,notHL,NaN,male,CHS_HOPKINS,CHS_Original
5528,203847,CHS_203847,CHS,White,notHL,NaN,male,CHS_DAVIS,CHS_Original
5529,405396,CHS_405396,CHS,White,notHL,NaN,female,CHS_PITT,CHS_Original


#### WHI

In [17]:
dg_dir = "/proj/yunligrp/users/raffield/TOPMED/Hematology_variants/ABOvariants/freeze5/topmed_dcc_demographic_v1_phs001237_whi"
dg_filename = "topmed_dcc_demographic_v1.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
race_summary_df, _ = categorize_df(dg, "race_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "ethnicity_1")
display(ethnicity_summary_df)

,race_1,sample_size
2,AI_AN,657
1,Asian,3573
6,Black,12318
0,HI_PI,111
3,Missing Values,86
4,Other,2888
5,White,123580


,ethnicity_1,sample_size
1,HL,5468
2,Missing Values,348
0,notHL,137397


In [19]:
dg_useful = dg[["unique_subject_key", "race_1", "ethnicity_1"]]
dg_useful.rename(columns = {"race_1":"race", "ethnicity_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1, "Asian":3, "AI_AN":4, "Other":6, "HI_PI":5, "HL":1, "notHL":0}, inplace = True)
sample_num = dg_useful.shape[0]
for sample_i in range(sample_num):
    if pd.isnull(dg_useful.loc[sample_i, "race"]) == True and dg_useful.loc[sample_i, "ethnicity"] == 1:
        dg_useful.loc[sample_i, "race"] = 0
    elif pd.isnull(dg_useful.loc[sample_i, "race"]) == False and pd.isnull(dg_useful.loc[sample_i, "ethnicity"]) == True:
        dg_useful.loc[sample_i, "ethnicity"] = 0

print(dg_useful.shape)
dg_useful.dropna(axis = 0, how = "any", inplace = True)
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "WHI_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

(143213, 3)


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,unique_subject_key,race,ethnicity
0,WHI_703256,1.0,1.0
1,WHI_707423,1.0,1.0
2,WHI_860306,6.0,1.0
3,WHI_779251,2.0,1.0
4,WHI_783710,1.0,1.0
...,...,...,...
143206,WHI_819195,1.0,0.0
143208,WHI_792045,1.0,0.0
143209,WHI_751658,1.0,0.0
143211,WHI_795430,2.0,0.0


In [18]:
display(dg)

,SUBJECT_ID,unique_subject_key,hispanic_subgroup_1,ethnicity_1,annotated_sex_1,geographic_site_1,race_1,topmed_abbreviation
0,703256,WHI_703256,Mexican,HL,female,WHI_West,White,WHI
1,707423,WHI_707423,Mexican,HL,female,WHI_South,White,WHI
2,860306,WHI_860306,Mexican,HL,female,WHI_South,Other,WHI
3,779251,WHI_779251,PuertoRican,HL,female,WHI_Northeast,Black,WHI
4,783710,WHI_783710,Mexican,HL,female,WHI_South,White,WHI
...,...,...,...,...,...,...,...,...
143208,792045,WHI_792045,NaN,NaN,female,WHI_West,White,WHI
143209,751658,WHI_751658,NaN,NaN,female,WHI_West,White,WHI
143210,844056,WHI_844056,NaN,NaN,female,WHI_Midwest,NaN,WHI
143211,795430,WHI_795430,NaN,NaN,female,WHI_Northeast,Black,WHI


#### MESA

In [192]:
dg_dir = "/proj/yunligrp/dbGAP_topmed/longleaf/dbGaP-12526/76121/topmed-dcc/exchange/phs001416_TOPMed_WGS_MESA/Phenotype/DCC/official/topmed_dcc_harmonized_demographic_v4_phs001416"
dg_filename = "topmed_dcc_harmonized_demographic_v4.txt"
dg_dir_filename = os.path.join(dg_dir, dg_filename)
dg = pd.read_csv(dg_dir_filename, sep = "\t", comment='#', header = 0, index_col = None)

In [45]:
race_summary_df, _ = categorize_df(dg, "race_us_1")
display(race_summary_df)
ethnicity_summary_df, _ = categorize_df(dg, "hispanic_or_latino_1")
display(ethnicity_summary_df)

,race_us_1,sample_size
0,Asian,777
1,Black,2654
2,Multiple,2
3,Other,2177
4,White,2686


,hispanic_or_latino_1,sample_size
2,HL,2177
0,Missing Values,5200
1,notHL,919


In [193]:
dg_useful = dg[["unique_subject_key", "race_us_1", "hispanic_or_latino_1"]]
dg_useful.rename(columns = {"race_us_1":"race", "hispanic_or_latino_1":"ethnicity"}, inplace = True)
dg_useful.replace(to_replace = {"Black":2, "White":1, "Asian":3, "Multiple":7, "Other":0, "HL":1, "notHL":0}, inplace = True)
sample_num = dg_useful.shape[0]
for sample_i in range(sample_num):
    if pd.isnull(dg_useful.loc[sample_i, "ethnicity"]) == True:
#         display(dg_useful.iloc[sample_i, :])
        dg_useful.loc[sample_i, "ethnicity"] = 0

print(dg_useful.shape)
dg_useful.dropna(axis = 0, how = "any", inplace = True)
display(dg_useful)
dg_useful_dir = os.path.join("..", "raw_data", "race_ethnicity")
dg_useful_filename = "MESA_race_ethnicity.tsv"
dg_useful_dir_filename = os.path.join(dg_useful_dir, dg_useful_filename)
dg_useful.to_csv(dg_useful_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

(8296, 3)


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,unique_subject_key,race,ethnicity
0,MESA_37066,0,1.0
1,MESA_14068,0,1.0
2,MESA_49033,0,1.0
3,MESA_13099,0,1.0
4,MESA_12582,0,1.0
...,...,...,...
8291,MESA_38668,2,0.0
8292,MESA_37106,2,0.0
8293,MESA_49007,1,0.0
8294,MESA_26398,2,0.0


In [20]:
display(dg)

,SUBJECT_ID,unique_subject_key,topmed_study,hispanic_subgroup_1,unit_hispanic_subgroup_1,annotated_sex_1,unit_annotated_sex_1,geographic_site_1,unit_geographic_site_1,subcohort_1,unit_subcohort_1,hispanic_or_latino_1,unit_hispanic_or_latino_1,race_us_1,unit_race_us_1
0,37066,MESA_37066,MESA,Mexican,MESA_Family,male,MESA_Family,MESA_UMN,MESA_Family,MESA_Family,MESA_family,HL,MESA_Family,Other,MESA_Family
1,14068,MESA_14068,MESA,Mexican,MESA_Classic,female,MESA_Classic,MESA_UCLA,MESA_Classic,MESA_Classic,MESA_classic,HL,MESA_Classic,Other,MESA_Classic
2,49033,MESA_49033,MESA,Dominican,MESA_AirNR,male,MESA_AirNR,MESA_COL,MESA_AirNR,MESA_AirNR,MESA_airnr,HL,MESA_AirNR,Other,MESA_AirNR
3,13099,MESA_13099,MESA,Dominican,MESA_Classic,male,MESA_Classic,MESA_COL,MESA_Classic,MESA_Classic,MESA_classic,HL,MESA_Classic,Other,MESA_Classic
4,12582,MESA_12582,MESA,PuertoRican,MESA_Classic,male,MESA_Classic,MESA_COL,MESA_Classic,MESA_Classic,MESA_classic,HL,MESA_Classic,Other,MESA_Classic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8291,38668,MESA_38668,MESA,NaN,NaN,female,MESA_Family,MESA_NWU,MESA_Family,MESA_Family,MESA_family,notHL,MESA_Family,Black,MESA_Family
8292,37106,MESA_37106,MESA,NaN,NaN,female,MESA_Family,MESA_NWU,MESA_Family,MESA_Family,MESA_family,notHL,MESA_Family,Black,MESA_Family
8293,49007,MESA_49007,MESA,NaN,NaN,male,MESA_AirNR,MESA_COL,MESA_AirNR,MESA_AirNR,MESA_airnr,NaN,NaN,White,MESA_AirNR
8294,26398,MESA_26398,MESA,NaN,NaN,female,MESA_Classic,MESA_NWU,MESA_Classic,MESA_Classic,MESA_classic,NaN,NaN,Black,MESA_Classic


#### FHS (Framingham)

In [74]:
btc_dir = "/proj/yunligrp/dbGAP_topmed/dbGaP-12526/55460/topmed-dcc/exchange/phs000974_TOPMed_WGS_Framingham/Phenotype/DCC/official/topmed_dcc_blood_cell_count_20170414_phs000974"
btc_filename = "topmed_dcc_blood_cell_count_20170414.txt"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", comment='#', header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

In [75]:
btc_prepared = btc.copy()
btc_prepared["cohort"] = "FHS"
btc_prepared["unique_subject_key"] = btc_prepared["cohort"] + "_" + btc_prepared["SUBJECT_ID"]
btc_prepared_dir = os.path.join("..", "raw_data", "phenotype")
btc_prepared_filename = "FHS_blood_cell_traits.tsv"
btc_prepared_dir_filename = os.path.join(btc_prepared_dir, btc_prepared_filename)
btc_prepared.to_csv(btc_prepared_dir_filename, sep = "\t", header = True, index = False)

#### Amish

In [79]:
btc_dir = "/proj/yunligrp/dbGAP_topmed/dbGaP-12526/55459/topmed-dcc/exchange/phs000956_TOPMed_WGS_Amish/Phenotype/DCC/official/topmed_dcc_blood_cell_count_20170414_phs000956"
btc_filename = "topmed_dcc_blood_cell_count_20170414.txt"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", comment='#', header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

In [80]:
btc_prepared = btc.copy()
btc_prepared["cohort"] = "Amish"
btc_prepared["unique_subject_key"] = btc_prepared["cohort"] + "_" + btc_prepared["SUBJECT_ID"]
btc_prepared_dir = os.path.join("..", "raw_data", "phenotype")
btc_prepared_filename = "Amish_blood_cell_traits.tsv"
btc_prepared_dir_filename = os.path.join(btc_prepared_dir, btc_prepared_filename)
btc_prepared.to_csv(btc_prepared_dir_filename, sep = "\t", header = True, index = False)

#### Appending JHS ln(wbc) and to phenotype

In [93]:
pheno_dir = os.path.join("..", "raw_data", "phenotype")
jhs_pheno_filename = "jhs_basic_phenotypes_05072019.txt"
jhs_pheno_dir_filename = os.path.join(pheno_dir, jhs_pheno_filename)
jhs_pheno = pd.read_csv(jhs_pheno_dir_filename, sep = "\t", header = 0, index_col = None)

In [51]:
jhs_pheno["neutrophil_ncnc_bld_1"] = np.exp(jhs_pheno["lnneu"])
jhs_pheno["age_at_neutrophil_ncnc_bld_1"] = jhs_pheno["age"]
jhs_pheno["lymphocyte_ncnc_bld_1"] = np.exp(jhs_pheno["lnlym"])
jhs_pheno["age_at_lymphocyte_ncnc_bld_1"] = jhs_pheno["age"]
jhs_pheno["basophil_ncnc_bld_1"] = np.exp(jhs_pheno["lnbaso"])
jhs_pheno["age_at_basophil_ncnc_bld_1"] = jhs_pheno["age"]
jhs_pheno["eosinophil_ncnc_bld_1"] = np.exp(jhs_pheno["lneos"])
jhs_pheno["age_at_eosinophil_ncnc_bld_1"] = jhs_pheno["age"]
jhs_pheno["monocyte_ncnc_bld_1"] = np.exp(jhs_pheno["lnmono"])
jhs_pheno["age_at_monocyte_ncnc_bld_1"] = jhs_pheno["age"]
jhs_pheno["wbc_ncnc_bld_1"] = np.exp(jhs_pheno["lnwbc"])
jhs_pheno["age_at_wbc_ncnc_bld_1"] = jhs_pheno["age"]
# jhs_pheno["DDIMER"] = np.exp(jhs_pheno["lnddimer"])
# jhs_pheno["age_at_DDIMER"] = jhs_pheno["age"]

In [53]:
useful_pheno_list = ["NWDID", "neutrophil_ncnc_bld_1", "age_at_neutrophil_ncnc_bld_1","lymphocyte_ncnc_bld_1",
                     "age_at_lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "age_at_basophil_ncnc_bld_1","eosinophil_ncnc_bld_1",
                     "age_at_eosinophil_ncnc_bld_1", "monocyte_ncnc_bld_1", "age_at_monocyte_ncnc_bld_1","wbc_ncnc_bld_1", "age_at_wbc_ncnc_bld_1"]
jhs_pheno_useful = jhs_pheno[useful_pheno_list]

In [55]:
jhs_useful_filename = "jhs_usefule_phenotype_05072019.tsv"
jhs_useful_dir_filename = os.path.join(pheno_dir, jhs_useful_filename)
jhs_pheno_useful.to_csv(jhs_useful_dir_filename, sep = "\t", header = True, index = False)

#### Adding Blood Cell Traits to original phenotype file

In [97]:
pheno_dir = os.path.join("..", "raw_data", "phenotype")
jhs_useful_filename = "jhs_usefule_phenotype_05072019.tsv"
jhs_useful_dir_filename = os.path.join(pheno_dir, jhs_useful_filename)
jhs_pheno_useful = pd.read_csv(jhs_useful_dir_filename, sep = "\t", header = 0, index_col = None)

In [88]:
btc_dir = os.path.join("..", "raw_data", "phenotype")
btc_filename = "btc01-coh03.tsv"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

In [98]:
anno_dir = os.path.join("..", "raw_data", "annotation")
anno_filename = "freeze8_anno05_af02_unique02.tsv"
anno_dir_filename = os.path.join(anno_dir, anno_filename)
anno = pd.read_csv(anno_dir_filename, sep = "\t", header = 0, index_col = None)

In [134]:
btc_jhs = btc.loc[btc.loc[:, "cohort"] == "JHS", :]
btc_nojhs = btc.loc[btc.loc[:, "cohort"] != "JHS", :]
# get the unique subject key of jhs_pheno_useful
jhs_pheno_useful_usk = anno[["NWDID", "unique_subject_key"]].merge(jhs_pheno_useful, on = "NWDID", how = "inner").drop(columns = ["NWDID"], inplace = False)
btc_nojhs_dir = os.path.join("..", "raw_data", "phenotype")
btc_nojhs_filename = "btc01-coh03.tsv"
btc_nojhs_dir_filename = os.path.join(btc_nojhs_dir, btc_nojhs_filename)
btc_nojhs.to_csv(btc_nojhs_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

In [133]:
df1 = btc_jhs.reset_index(drop = True)
df2 = jhs_pheno_useful_usk
common_col = ["unique_subject_key"]
fillin_col_raw = list(df2)
fillin_col = [x for x in fillin_col_raw if x not in common_col]
df1_fillin = df1.combine_first(df1.drop(columns = fillin_col).merge(df2, how = 'left', on = common_col))
df1_fillin_dir = os.path.join("..", "raw_data", "phenotype")
df1_fillin_filename = "JHS_blood_cell_traits.tsv"
df1_fillin_dir_filename = os.path.join(df1_fillin_dir, df1_fillin_filename)
df1_fillin.to_csv(df1_fillin_dir_filename, sep = "\t", header = True, index = False, quoting=csv.QUOTE_NONE)

### Concatenating Blood Cell Traits

In [136]:
load_dir = os.path.join("..", "raw_data", "phenotype")
aric_filename = "ARIC_blood_cell_traits.tsv"
aric_dir_filename = os.path.join(load_dir, aric_filename)
aric = pd.read_csv(aric_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

safs_filename = "SAFS_blood_cell_traits.tsv"
safs_dir_filename = os.path.join(load_dir, safs_filename)
safs = pd.read_csv(safs_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

fhs_filename = "FHS_blood_cell_traits.tsv"
fhs_dir_filename = os.path.join(load_dir, fhs_filename)
fhs = pd.read_csv(fhs_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

amish_filename = "Amish_blood_cell_traits.tsv"
amish_dir_filename = os.path.join(load_dir, amish_filename)
amish = pd.read_csv(amish_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

jhs_filename = "JHS_blood_cell_traits.tsv"
jhs_dir_filename = os.path.join(load_dir, jhs_filename)
jhs = pd.read_csv(jhs_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

In [137]:
btc_dir = os.path.join("..", "raw_data", "phenotype")
btc_filename = "btc01-coh03.tsv"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", header = 0, index_col = None, dtype={"SUBJECT_ID":"string"})

In [139]:
print(btc.shape, aric.shape, safs.shape, fhs.shape, amish.shape, jhs.shape)

(188876, 35) (14875, 33) (1494, 33) (8065, 33) (1116, 33) (3406, 35)


In [141]:
# common_col = list(aric)
btc_aric_safs_fhs_amish_jhs = pd.concat([btc, aric, safs, fhs, amish, jhs], axis = 0, join = "outer")
btc_aric_safs_fhs_amish_jhs_dir = os.path.join("..", "raw_data", "phenotype")
btc_aric_safs_fhs_amish_jhs_filename = "btc01-coh04.tsv"
btc_aric_safs_fhs_amish_jhs_dir_filename = os.path.join(btc_aric_safs_fhs_amish_jhs_dir, btc_aric_safs_fhs_amish_jhs_filename)
btc_aric_safs_fhs_amish_jhs.to_csv(btc_aric_safs_fhs_amish_jhs_dir_filename, sep = "\t", header = True, index = False)

### Set outlier values as NA

In [165]:
btc_dir = os.path.join("..", "raw_data", "phenotype")
btc_filename = "btc01-coh04.tsv"
btc_dir_filename = os.path.join(btc_dir, btc_filename)
btc = pd.read_csv(btc_dir_filename, sep = "\t", header = 0, index_col = None, dtype = {"SUBJECT_ID":"string"})

In [173]:
btc_traits_list = ['hemoglobin_mcnc_bld_1', 'hematocrit_vfr_bld_1', 'rbc_ncnc_bld_1', 'mcv_entvol_rbc_1', 'mch_entmass_rbc_1',
                   'mchc_mcnc_rbc_1', 'rdw_ratio_rbc_1', 'neutrophil_ncnc_bld_1', 'lymphocyte_ncnc_bld_1', 'basophil_ncnc_bld_1',
                   'eosinophil_ncnc_bld_1', 'monocyte_ncnc_bld_1', 'wbc_ncnc_bld_1', 'pmv_entvol_bld_1', 'platelet_ncnc_bld_1']
all_header = list(btc)
other_header = [x for x in all_header if x not in btc_traits_list]

btc_trait = btc[btc_traits_list]
btc_not_trait = btc[other_header]

btc_trait.replace(to_replace = {0:np.nan}, inplace = True)
btc_trait.loc[btc_trait.loc[:, "platelet_ncnc_bld_1"] > 1000, "platelet_ncnc_bld_1"] = np.nan
btc_trait.loc[btc_trait.loc[:, "hematocrit_vfr_bld_1"] > 60, "hematocrit_vfr_bld_1"] = np.nan
btc_trait.loc[btc_trait.loc[:, "hemoglobin_mcnc_bld_1"] > 20, "hemoglobin_mcnc_bld_1"] = np.nan
btc_trait.loc[btc_trait.loc[:, "wbc_ncnc_bld_1"] > 200, "wbc_ncnc_bld_1"] = np.nan

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [175]:
btc_rm = pd.concat([btc_not_trait, btc_trait], axis = 1)
btc_rm_dir = os.path.join("..", "raw_data", "phenotype")
btc_rm_filename = "btc01-coh04.tsv"
btc_rm_dir_filename = os.path.join(btc_rm_dir, btc_rm_filename)
btc_rm.to_csv(btc_rm_dir_filename, sep = "\t", header = True, index = False)

## Race Ethnicity Demographic Data

In [20]:
load_dir = os.path.join("..", "raw_data", "race_ethnicity")
cohort_list = ["HCHS_SOL", "BioMe", "WHI", "CHS", "MESA", "CARDIA", "GeneSTAR", "COPDGene", "ARIC"]
race_ethnicity_dir_filename_list = [os.path.join(load_dir, f"{cohort}_race_ethnicity.tsv") for cohort in cohort_list]
race_ethnicity_list = read2df_list(race_ethnicity_dir_filename_list)

In [21]:
race_ethnicity = pd.concat(race_ethnicity_list, axis = 0)

In [22]:
race_ethnicity_dir = os.path.join("..", "raw_data", "race_ethnicity")
race_ethnicity_filename = "race_ethnicity.tsv"
race_ethnicity_dir_filename = os.path.join(race_ethnicity_dir, race_ethnicity_filename)
race_ethnicity.to_csv(race_ethnicity_dir_filename, sep = "\t", header = True, index = False)